### Import Modules

In [47]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [49]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter/special_functions.pyc'>

### Specify fit parameters

In [28]:
p_dict = {
    'order' : {
        'fit' : 'nlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'xpt-taylor', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    
    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [29]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if False: #p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
100% complete Time (s):  0.372256994247
Compiling results...
Done.

Total time (s):  5.73826003075 



In [22]:
myfit = bootstrapper.fits[0]

In [25]:
myfit.psdev

BufferDict([('mpi', array([6.65491974e-05, 1.18907520e-04, 1.94264299e-04, 1.44152825e-04,
       2.47567722e-04, 1.74591039e-04, 3.16631509e-04, 1.68716994e-04,
       2.01975052e-04, 1.63326009e-04, 1.86459309e-04, 2.97297713e-04,
       2.89473383e-04, 3.03481609e-04])), ('mk', array([9.87188715e-05, 1.39753944e-04, 1.18677093e-04, 1.23954989e-04,
       2.27785883e-04, 1.70332816e-04, 2.12615307e-04, 2.07414419e-04,
       1.59972495e-04, 1.41836084e-04, 2.06529783e-04, 2.41664605e-04,
       2.64537372e-04, 2.64896696e-04])), ('mss', array([7.40163784e-05, 1.05929825e-04, 1.05298574e-04, 1.08070049e-04,
       1.05393558e-04, 1.16792382e-04, 1.43230670e-04, 1.58676720e-04,
       1.33100200e-04, 1.25779834e-04, 1.30922159e-04, 1.64787083e-04,
       2.21221492e-04, 2.20367015e-04])), ('mju', array([0.00026616, 0.00036636, 0.00067445, 0.00090075, 0.00131229,
       0.00087781, 0.00087781, 0.00090659, 0.00125692, 0.00159482,
       0.00212134, 0.00132647, 0.00348649, 0.00235276])), 

In [69]:
myfit.data

BufferDict([('xpt-taylor_nnlo', array([1.1434(28), 1.1101(24), 1.0994(23), 1.0810(22), 1.1600(52),
       1.1306(32), 1.1378(40), 1.1033(26), 1.0876(28), 1.0739(22),
       1.1263(19), 1.1015(19), 1.0884(19), 1.0713(20)], dtype=object))])

In [66]:
def test(self):
    temp_prior  = {}
    for key in self.get_fit_parameters().keys():
        #temp_prior[key] = gv.gvar(0, 0)
        if True: #key in ['L_5']:
            value = np.random.normal(gv.mean(self.get_fit_parameters(key)), gv.sdev(self.get_fit_parameters(key)))
            temp_prior[key] = value
        
    return self.fk_fpi_fit_fcn(fit_parameters=temp_prior)

test(bootstrapper)

0.9682(30)

In [76]:
myarray = [test(bootstrapper) for j in range(100)]
np.median(myarray)

1.1745(47)

In [116]:
def test(self):
    temp_data = self._make_fit_data(0)
    eps2_a = np.median((temp_data['a/w0'] / (4 *np.pi))**2)
    eps2_k = np.median((temp_data['mk'])**2 / temp_data['lam2_chi'])
    eps2_pi = np.median((temp_data['mpi'])**2 / temp_data['lam2_chi'])
    temp = self.fits[0].fcn_p
    fk_fpi = (np.median(temp[temp.keys()[0]]))
    
    for key in self.get_fit_parameters():
        value = 0
        if key == 'A_a':
            value = self.get_fit_parameters(key) *eps2_a
            
        elif key == 'A_k':
            value = self.get_fit_parameters(key) *eps2_k
            
        elif key == 'A_p':
            value = self.get_fit_parameters(key) *eps2_pi
        
        elif key == 'A_aa':
            value = self.get_fit_parameters(key) *eps2_a**2
            
        elif key == 'A_ak':
            value = self.get_fit_parameters(key) *eps2_a *eps2_k
            
        elif key == 'A_ap':
            value = self.get_fit_parameters(key) *eps2_a *eps2_pi
            
        elif key == 'A_kk':
            value = self.get_fit_parameters(key) *eps2_k**2
            
        elif key == 'A_kp':
            value = self.get_fit_parameters(key) *eps2_k *eps2_pi
            
        elif key == 'A_pp':
            value = self.get_fit_parameters(key) *eps2_pi**2
        
        value = value *(eps2_k - eps2_pi)
        if key not in ['L_4', 'L_5']:
            if ((np.abs(gv.mean(value)+gv.sdev(value)) < gv.sdev(fk_fpi)) 
                    and (np.abs(gv.mean(value)-gv.sdev(value)) < gv.sdev(fk_fpi))):
                print key, "is hopeless:", value, 'vs', gv.sdev(fk_fpi)
    
test(bootstrapper)

A_ap is hopeless: -0.00001(18) vs 0.00106834119412
A_ak is hopeless: -0.00004(53) vs 0.00106834119412
A_aa is hopeless: -2(1035)e-08 vs 0.00106834119412


### Or make all fits

In [50]:
p_dict = {
    'order' : {
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'

    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [51]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor']:#, 'ma-old']:
    for order_fit in ['nlo', 'nnlo']:#, 'nnnlo']:
        
        print fit_type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()
        
        #print test(bootstrapper)

        print "\nTotal time (s): ", t1 - t0, "\n"

xpt nlo
Using default prior.
Making fits...
100% complete Time (s):  0.528932094574
Compiling results...
Done.
Saving...
Done.

Total time (s):  7.46955084801 

xpt nnlo
Using default prior.
Making fits...
100% complete Time (s):  1.46111679077
Compiling results...
Done.
Saving...
Done.

Total time (s):  10.4104230404 

xpt-taylor nlo
Using default prior.
Making fits...
100% complete Time (s):  0.602602005005
Compiling results...
Done.
Saving...
Done.

Total time (s):  6.98909497261 

xpt-taylor nnlo
Using default prior.
Making fits...
100% complete Time (s):  1.10137295723
Compiling results...
Done.
Saving...
Done.

Total time (s):  7.74492096901 

ma nlo
Using default prior.
Making fits...
100% complete Time (s):  1.70426797867
Compiling results...
Done.
Saving...
Done.

Total time (s):  41.2194199562 

ma nnlo
Using default prior.
Making fits...
100% complete Time (s):  2.62448692322
Compiling results...
Done.
Saving...
Done.

Total time (s):  40.6861710548 

ma-taylor nlo
Using def

In [38]:
#bootstrapper.fits[0].show_plots(view='std')